In [146]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [147]:
import os
import numpy as np
import pandas as pd

import re

#pd.set_option('display.max_row',None)
pd.set_option('display.max_columns',None)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings('ignore',category=FutureWarning)

In [148]:
DATASET_ROOT = '/content/drive/MyDrive/機械学習/Competitions/Signate/SOTA [住宅予測]'
INPUT_DIR = os.path.join(DATASET_ROOT,'input')
train = pd.read_csv(os.path.join(INPUT_DIR,'train.csv'))
test = pd.read_csv(os.path.join(INPUT_DIR,'test.csv'))
sub = pd.read_csv(os.path.join(INPUT_DIR,'sample_submit.csv'))

In [149]:
for dirname,_,filenames in os.walk(INPUT_DIR):
  for filename in filenames:
    print(os.path.join(dirname,filename))

/content/drive/MyDrive/機械学習/Competitions/Signate/SOTA [住宅予測]/input/sample_submit.csv
/content/drive/MyDrive/機械学習/Competitions/Signate/SOTA [住宅予測]/input/test.csv
/content/drive/MyDrive/機械学習/Competitions/Signate/SOTA [住宅予測]/input/train.csv
/content/drive/MyDrive/機械学習/Competitions/Signate/SOTA [住宅予測]/input/Simple_DataFrame.csv
/content/drive/MyDrive/機械学習/Competitions/Signate/SOTA [住宅予測]/input/train_processed.csv
/content/drive/MyDrive/機械学習/Competitions/Signate/SOTA [住宅予測]/input/test_processed.csv
/content/drive/MyDrive/機械学習/Competitions/Signate/SOTA [住宅予測]/input/train_enginiring.csv
/content/drive/MyDrive/機械学習/Competitions/Signate/SOTA [住宅予測]/input/test_enginiring.csv


In [150]:
train_process = pd.read_csv(os.path.join(INPUT_DIR,'train_processed.csv'))
test_process = pd.read_csv(os.path.join(INPUT_DIR,'test_processed.csv'))
simple_df = pd.read_csv(os.path.join(INPUT_DIR,'Simple_DataFrame.csv'))
train_process.head()

,賃料,方角,建物構造,adr0,adr1,adr2,adr3,adr4,line0,station0,distance0,line1,station1,distance1,line2,station2,distance2,age,area,floorplan,withstorage,thisfloor,maxfloor,シャワー,トイレなし,バスなし,バス・トイレ別,共同トイレ,共同バス,専用トイレ,専用バス,洗面台独立,浴室乾燥機,温水洗浄便座,脱衣所,追焚機能,IHコンロ,L字キッチン,カウンターキッチン,ガスコンロ,コンロ1口,コンロ2口,コンロ3口,コンロ4口以上,コンロ設置可(コンロ1口),コンロ設置可(コンロ2口),コンロ設置可(コンロ3口),コンロ設置可(コンロ4口以上),コンロ設置可(口数不明),システムキッチン,冷蔵庫あり,独立キッチン,給湯,電気コンロ,BSアンテナ,CATV,CSアンテナ,インターネット使用料無料,インターネット対応,光ファイバー,有線放送,高速インターネット,24時間換気システム,2面採光,3面採光,ウォークインクローゼット,エアコン付,エレベーター,オール電化,ガスその他,ガス暖房,クッションフロア,シューズボックス,タイル張り,トランクルーム,バリアフリー,バルコニー,フローリング,プロパンガス,ペアガラス,ルーフバルコニー,ロフト付き,下水,二世帯住宅,二重サッシ,井戸,公営水道,冷房,出窓,地下室,室内洗濯機置場,室外洗濯機置場,専用庭,床下収納,床暖房,排水その他,敷地内ごみ置き場,水道その他,汲み取り,洗濯機置場なし,浄化槽,石油暖房,都市ガス,防音室,【クリーニング】,【コインパーキング】,【コンビニ】,【スーパー】,【デパート】,【ドラッグストア】,【レンタルビデオ】,【公園】,【図書館】,【大学】,【学校】,【小学校】,【幼稚園・保育園】,【月極駐車場】,【病院】,【総合病院】,【郵便局】,【銀行】,【飲食店】,park_bike,park_motor,park_car,contr
0,75000,南東,RC（鉄筋コンクリート）,東京都北区,滝野川,3.0,NaN,NaN,都営三田線,西巣鴨駅,4,都電荒川線,西ケ原四丁目駅,7.0,埼京線,板橋駅,14.0,9.750000,20.01,1K,0,1,12.0,1,0,0,1,0,0,1,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,2048,2048,588,311,2048,956,2048,1103,2048,461,2048,495,2048,2048,2048,2048,1246,2048,378,1,0,0,2.0
1,76000,NaN,鉄骨造,東京都中央区,月島,3.0,NaN,NaN,都営大江戸線,勝どき駅,5,有楽町線,月島駅,9.0,日比谷線,築地駅,20.0,44.833333,16.50,1R,0,5,10.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,2048,2048,2048,1283,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,1,0,0,2.0
2,110000,南,RC（鉄筋コンクリート）,東京都渋谷区,笹塚,2.0,NaN,NaN,京王線,笹塚駅,6,京王線,代田橋駅,7.0,京王線,明大前駅,17.0,8.500000,22.05,1K,0,12,15.0,1,0,0,1,0,0,1,1,1,1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,2048,2048,184,89,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,1,1,2,2.0
3,150000,南,RC（鉄筋コンクリート）,東京都杉並区,高円寺南,2.0,23.0,2.0,丸ノ内線(池袋－荻窪),新高円寺駅,3,総武線・中央線（各停）,高円寺駅,9.0,丸ノ内線(池袋－荻窪),東高円寺駅,14.0,29.333333,60.48,2LDK,0,3,4.0,1,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,2048,2048,326,225,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,0,0,0,2.0
4,74000,南,木造,東京都葛飾区,金町,3.0,7.0,2.0,京成金町線,京成金町駅,5,常磐線,金町駅,7.0,京成金町線,柴又駅,17.0,31.583333,39.66,2DK,0,1,2.0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,9,9,9,9,9,9,9,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2048,2048,351,193,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,0,0,2,2.0


In [151]:
print(f'train row is {train_process.shape[0]}. train col is {train_process.shape[1]}. train miss is {train_process.isnull().sum().sum()}')
print(f'test row is {test_process.shape[0]}. test col is {test_process.shape[1]}. test miss is {test_process.isnull().sum().sum()}')

train row is 31470. train col is 127. train miss is 64076
test row is 31262. test col is 126. test miss is 64451


In [152]:
train_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31470 entries, 0 to 31469
Columns: 127 entries, 賃料 to contr
dtypes: float64(9), int64(107), object(11)
memory usage: 30.5+ MB


In [153]:
train_process['設備_sum'] = train_process.loc[:,'シャワー':'防音室'].sum(axis=1)
test_process['設備_sum'] = test_process.loc[:,'シャワー':'防音室'].sum(axis=1)

In [154]:
catg_feats = train_process.dtypes[train_process.dtypes=='object'].index
print(f'count categorical features {len(catg_feats)}')
print('='*30)
print(catg_feats)

count categorical features 11
Index(['方角', '建物構造', 'adr0', 'adr1', 'line0', 'station0', 'line1', 'station1',
       'line2', 'station2', 'floorplan'],
      dtype='object')


In [155]:
df = pd.concat([train_process,test_process],axis=0).reset_index(drop=True)
print(train_process.shape[0],test_process.shape[0],df.shape)

31470 31262 (62732, 128)


In [156]:
!pip install category_encoders

In [157]:
#エンコーディング(ターゲット)
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

In [158]:
simple_df.head()

,id,Place,Access,Room,Passed,Angle,Area,Floor,Bath,Kitchen,Internet,Facility,Parking,Building,Material,Contract,Place_countall,Access_countall,Room_countall,Passed_countall,Angle_countall,Area_countall,Floor_countall,Bath_countall,Kitchen_countall,Internet_countall,Facility_countall,Parking_countall,Building_countall,Material_countall,Contract_countall,Strage_Room,count_Room,Place_ku,Place_ku_countall,Num_of_Null,Building_element_count,Facility_element_count,Kitchen_element_count,Bath_element_count,Internet_element_count,Contract_imp,Passed_year,Passed_month,Passed_all_month,Passed_year_Countall,Building_floor,Building_story,Have_UnderGround,Building_floor_Countall,Building_story_Countall,Have_Parking,Parking_yen,Parking_meter,Have_Bike,Bike_yen,Bike_meter,Have_Cycle,Cycle_yen,Cycle_meter,parking_sum_yen,parking_sum_meter,parking_sum_exist,Label_Place,Label_Room,Label_Angle,Label_Place_ku,Label_Material
0,1,東京都北区滝野川３丁目,都営三田線\t西巣鴨駅\t徒歩4分\t\t埼京線\t板橋駅\t徒歩14分\t\t都電荒川線\...,K,9年9ヶ月,南東,20.01,1階／12階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄便座,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCATV／\tCSアンテナ／\tBSアンテナ,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有,【小学校】 495m\t【大学】 461m\t【小学校】 962m\t【公園】 1103m\...,RC（鉄筋コンクリート）,2年間,18,1,23661,78,6150,73,16,1137,11953,1390,819,2473,1,34810,45052,0,1,東京都北区,2061,0,10.0,11.0,5.0,7.0,4.0,2,9,9,117,1062,1,12,0,12032,2631,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,0.0,0.0,0.0,2.0,3224,1,4,3,3
1,2,東京都中央区月島３丁目,都営大江戸線\t勝どき駅\t徒歩5分\t\t有楽町線\t月島駅\t徒歩9分\t\t日比谷線\...,R,44年10ヶ月,NaN,16.50,5階／10階建,専用トイレ／\tシャワー／\t温水洗浄便座,ガスコンロ／\tシステムキッチン\t／\t給湯,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t室内洗濯機置場／\t敷...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【スーパー】 1283m,鉄骨造,2年間,44,1,10178,37,5557,144,359,92,1116,5166,106,4853,1,7495,45052,0,1,東京都中央区,2532,1,1.0,9.0,4.0,3.0,1.0,2,44,10,538,283,5,10,0,3946,2978,0.0,NaN,NaN,0.0,NaN,NaN,2.0,NaN,0.0,0.0,0.0,2.0,2215,4,8,1,10
2,3,東京都渋谷区笹塚２丁目,京王線\t笹塚駅\t徒歩6分\t\t京王線\t代田橋駅\t徒歩7分\t\t京王線\t明大前駅...,K,8年6ヶ月,南,22.05,12階／15階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\t光ファイバー／\tCSアンテナ／\tBSアンテナ,エアコン付\tウォークインクローゼット\tシューズボックス／\tバルコニー／\tフローリング...,"駐輪場\t空有\tバイク置き場\t空有\t駐車場\t近隣\t30,000円(税込)\t距離100m",【スーパー】 89m\t【コンビニ】 184m\t【コンビニ】 392m\t【スーパー】 492m,RC（鉄筋コンクリート）,2年間,66,3,23661,131,17697,32,100,5785,11953,2041,259,37,3,34810,45052,0,1,東京都渋谷区,1978,0,4.0,13.0,5.0,9.0,4.0,2,8,6,102,992,12,15,0,661,1273,1.0,30000.0,100.0,2.0,NaN,0.0,2.0,NaN,0.0,30000.0,100.0,5.0,11534,1,3,15,3
3,4,東京都杉並区高円寺南２丁目23-2,総武線・中央線（各停）\t高円寺駅\t徒歩9分\t\t丸ノ内線(池袋－荻窪)\t新高円寺駅\...,LDK,29年4ヶ月,南,60.48,3階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座／\t洗面台独立,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー,エアコン付\tシューズボックス／\tバルコニー／\t2面採光／\t室内洗濯機置場／\tエレベ...,駐車場\t無\t駐輪場\t無\tバイク置き場\t無,【スーパー】 225m\t【スーパー】 448m\t【スーパー】 619m\t【スーパー】 ...,RC（鉄筋コンクリート）,2年間\t※この物件は\t定期借家\tです。,1,1,5131,202,17697,7,1739,487,1885,3122,16,3603,1,34810,1888,0,2,東京都杉並区,3542,0,6.0,9.0,2.0,6.0,2.0,2,29,4,352,1580,3,4,0,10331,6165,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,8798,2,3,11,3
4,5,東京都葛飾区金町３丁目7-2,京成金町線\t京成金町駅\t徒歩5分\t\t常磐線\t金町(東京都)駅\t徒歩7分\t\t京...,DK,31年7ヶ月,南,39.66,1階／2階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能,給湯／\t独立キッチン,NaN,バルコニー／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水,"駐車場\t近隣\t17,000円(税込)\t距離300m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 193m\t【スーパー】 298m\t【スーパー】 660m\t【スーパー】 ...,木造,2年間,1,1,3715,141,17697,49,5519,390,29,6894,10,9,1,11062,45052,0,2,東京都葛飾区,2387,1,5.0,5.0,2.0,5.0,0.0,2,31,7,379,1328,1,2,0,12032,12312,1.0,17000.0,300.0,0.0,NaN,NaN,0.0,NaN,NaN,17000.0,300.0,1.0,15221,0,3,20,7


In [159]:
simple_df['Floor_count'] = simple_df['Room'].str.len()

In [160]:
simple_df['Floor_total'] = simple_df.Floor_count + simple_df.count_Room

In [161]:
Count_df = simple_df.copy()
Count_df = Count_df.loc[:,'Place_countall':]

In [162]:
Count_df.drop('Place_ku',axis=1,inplace=True)

In [163]:
Count_df['id'] = simple_df['id']
Count_df['Room'] = simple_df['Room']

In [164]:
Count_df.drop(['Strage_Room','Passed_all_month','Building_floor','Building_story','Passed_year','Passed_month'],axis=1,inplace=True)

In [165]:
Count_df.head()

,Place_countall,Access_countall,Room_countall,Passed_countall,Angle_countall,Area_countall,Floor_countall,Bath_countall,Kitchen_countall,Internet_countall,Facility_countall,Parking_countall,Building_countall,Material_countall,Contract_countall,count_Room,Place_ku_countall,Num_of_Null,Building_element_count,Facility_element_count,Kitchen_element_count,Bath_element_count,Internet_element_count,Contract_imp,Passed_year_Countall,Have_UnderGround,Building_floor_Countall,Building_story_Countall,Have_Parking,Parking_yen,Parking_meter,Have_Bike,Bike_yen,Bike_meter,Have_Cycle,Cycle_yen,Cycle_meter,parking_sum_yen,parking_sum_meter,parking_sum_exist,Label_Place,Label_Room,Label_Angle,Label_Place_ku,Label_Material,Floor_count,Floor_total,id,Room
0,18,1,23661,78,6150,73,16,1137,11953,1390,819,2473,1,34810,45052,1,2061,0,10.0,11.0,5.0,7.0,4.0,2,1062,0,12032,2631,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,0.0,0.0,0.0,2.0,3224,1,4,3,3,1,2,1,K
1,44,1,10178,37,5557,144,359,92,1116,5166,106,4853,1,7495,45052,1,2532,1,1.0,9.0,4.0,3.0,1.0,2,283,0,3946,2978,0.0,NaN,NaN,0.0,NaN,NaN,2.0,NaN,0.0,0.0,0.0,2.0,2215,4,8,1,10,1,2,2,R
2,66,3,23661,131,17697,32,100,5785,11953,2041,259,37,3,34810,45052,1,1978,0,4.0,13.0,5.0,9.0,4.0,2,992,0,661,1273,1.0,30000.0,100.0,2.0,NaN,0.0,2.0,NaN,0.0,30000.0,100.0,5.0,11534,1,3,15,3,1,2,3,K
3,1,1,5131,202,17697,7,1739,487,1885,3122,16,3603,1,34810,1888,2,3542,0,6.0,9.0,2.0,6.0,2.0,2,1580,0,10331,6165,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,8798,2,3,11,3,3,5,4,LDK
4,1,1,3715,141,17697,49,5519,390,29,6894,10,9,1,11062,45052,2,2387,1,5.0,5.0,2.0,5.0,0.0,2,1328,0,12032,12312,1.0,17000.0,300.0,0.0,NaN,NaN,0.0,NaN,NaN,17000.0,300.0,1.0,15221,0,3,20,7,2,4,5,DK


In [166]:
Count_df['Have_Parking'] = Count_df['Have_Parking'].fillna(2)
Count_df['Have_Bike'] = Count_df['Have_Bike'].fillna(2)
Count_df['Have_Cycle'] = Count_df['Have_Cycle'].fillna(2)
Count_df['Parking_meter'] = Count_df['Parking_meter'].fillna(0)
Count_df['Parking_yen'] = Count_df['Parking_yen'].fillna(0)
Count_df['Bike_meter'] = Count_df['Bike_meter'].fillna(0)
Count_df['Bike_yen'] = Count_df['Bike_yen'].fillna(0)
Count_df['Cycle_meter'] = Count_df['Cycle_meter'].fillna(0)
Count_df['Cycle_yen'] = Count_df['Cycle_yen'].fillna(0)

In [167]:
df['id'] = simple_df['id']

In [168]:
sample = pd.concat([df,Count_df],axis=1)

In [169]:
sample.head()

,賃料,方角,建物構造,adr0,adr1,adr2,adr3,adr4,line0,station0,distance0,line1,station1,distance1,line2,station2,distance2,age,area,floorplan,withstorage,thisfloor,maxfloor,シャワー,トイレなし,バスなし,バス・トイレ別,共同トイレ,共同バス,専用トイレ,専用バス,洗面台独立,浴室乾燥機,温水洗浄便座,脱衣所,追焚機能,IHコンロ,L字キッチン,カウンターキッチン,ガスコンロ,コンロ1口,コンロ2口,コンロ3口,コンロ4口以上,コンロ設置可(コンロ1口),コンロ設置可(コンロ2口),コンロ設置可(コンロ3口),コンロ設置可(コンロ4口以上),コンロ設置可(口数不明),システムキッチン,冷蔵庫あり,独立キッチン,給湯,電気コンロ,BSアンテナ,CATV,CSアンテナ,インターネット使用料無料,インターネット対応,光ファイバー,有線放送,高速インターネット,24時間換気システム,2面採光,3面採光,ウォークインクローゼット,エアコン付,エレベーター,オール電化,ガスその他,ガス暖房,クッションフロア,シューズボックス,タイル張り,トランクルーム,バリアフリー,バルコニー,フローリング,プロパンガス,ペアガラス,ルーフバルコニー,ロフト付き,下水,二世帯住宅,二重サッシ,井戸,公営水道,冷房,出窓,地下室,室内洗濯機置場,室外洗濯機置場,専用庭,床下収納,床暖房,排水その他,敷地内ごみ置き場,水道その他,汲み取り,洗濯機置場なし,浄化槽,石油暖房,都市ガス,防音室,【クリーニング】,【コインパーキング】,【コンビニ】,【スーパー】,【デパート】,【ドラッグストア】,【レンタルビデオ】,【公園】,【図書館】,【大学】,【学校】,【小学校】,【幼稚園・保育園】,【月極駐車場】,【病院】,【総合病院】,【郵便局】,【銀行】,【飲食店】,park_bike,park_motor,park_car,contr,設備_sum,id,Place_countall,Access_countall,Room_countall,Passed_countall,Angle_countall,Area_countall,Floor_countall,Bath_countall,Kitchen_countall,Internet_countall,Facility_countall,Parking_countall,Building_countall,Material_countall,Contract_countall,count_Room,Place_ku_countall,Num_of_Null,Building_element_count,Facility_element_count,Kitchen_element_count,Bath_element_count,Internet_element_count,Contract_imp,Passed_year_Countall,Have_UnderGround,Building_floor_Countall,Building_story_Countall,Have_Parking,Parking_yen,Parking_meter,Have_Bike,Bike_yen,Bike_meter,Have_Cycle,Cycle_yen,Cycle_meter,parking_sum_yen,parking_sum_meter,parking_sum_exist,Label_Place,Label_Room,Label_Angle,Label_Place_ku,Label_Material,Floor_count,Floor_total,id,Room
0,75000.0,南東,RC（鉄筋コンクリート）,東京都北区,滝野川,3.0,NaN,NaN,都営三田線,西巣鴨駅,4,都電荒川線,西ケ原四丁目駅,7.0,埼京線,板橋駅,14.0,9.750000,20.01,1K,0,1.0,12.0,1,0,0,1,0,0,1,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,1,1,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,2048,2048,588,311,2048,956,2048,1103,2048,461,2048,495,2048,2048,2048,2048,1246,2048,378,1,0,0,2.0,24,1,18,1,23661,78,6150,73,16,1137,11953,1390,819,2473,1,34810,45052,1,2061,0,10.0,11.0,5.0,7.0,4.0,2,1062,0,12032,2631,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,3224,1,4,3,3,1,2,1,K
1,76000.0,NaN,鉄骨造,東京都中央区,月島,3.0,NaN,NaN,都営大江戸線,勝どき駅,5,有楽町線,月島駅,9.0,日比谷線,築地駅,20.0,44.833333,16.50,1R,0,5.0,10.0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,2048,2048,2048,1283,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,1,0,0,2.0,15,2,44,1,10178,37,5557,144,359,92,1116,5166,106,4853,1,7495,45052,1,2532,1,1.0,9.0,4.0,3.0,1.0,2,283,0,3946,2978,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2215,4,8,1,10,1,2,2,R
2,110000.0,南,RC（鉄筋コンクリート）,東京都渋谷区,笹塚,2.0,NaN,NaN,京王線,笹塚駅,6,京王線,代田橋駅,7.0,京王線,明大前駅,17.0,8.500000,22.05,1K,0,12.0,15.0,1,0,0,1,0,0,1,1,1,1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,2048,2048,184,89,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,1,1,2,2.0,28,3,66,3,23661,131,17697,32,100,5785,11953,2041,259,37,3,34810,45052,1,1978,0,4.0,13.0,5.0,9.0,4.0,2,992,0,661,1273,1.0,30000.0,100.0,2.0,0.0,0.0,2.0,0.0,0.0,30000.0,100.0,5.0,11534,1,3,15,3,1,2,3,K
3,150000.0,南,RC（鉄筋コンクリート）,東京都杉並区,高円寺南,2.0,23.0,2.0,丸ノ内線(池袋－荻窪),新高円寺駅,3,総武線・中央線（各停）,高円寺駅,9.0,丸ノ内線(池袋－荻窪),東高円寺駅,14.0,29.333333,60.48,2LDK,0,3.0,4.0,1,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,2048,2048,326,225,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,2048,0,0,0,2.0,19,4,1,1,5131,202,17697,7,1739,487,1885,3122,16,3603,1,34810,1888,2,3542,0,6.0,9.0,2.0,6.0,2.0,2,1580,0,10331,6165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8798,2,3,11,3,3,5,4,LDK
4,74000.0,南,木造,東京都葛飾区,金町,3.0,7.0,2.0,京成金町線,京成金町駅,5

In [170]:
#ラベルエンコーディング、カウントエンコーディング、ターゲットエンコーディング
#クリーニング～飲食店の統計量
#組み合わせ

In [171]:
cols = sample.loc[:,'【クリーニング】':'【飲食店】'].columns
sample['distance_mean'] = sample[cols].mean(axis=1)
sample['distance_std'] = sample[cols].std(axis=1)
sample['distance_max'] = sample[cols].max(axis=1)
sample['distance_min'] = sample[cols].min(axis=1)

In [172]:
value_2048 = (sample[cols]==2048)
sample['distance2048'] = value_2048.sum(axis=1)

In [173]:
from sklearn.preprocessing import OrdinalEncoder

In [174]:
label_list = ['adr1','line0','station0','line1','station1','line2','station2']

for col in label_list:
  le = LabelEncoder()
  le.fit(sample[col].values)
  sample['Label_'+col] = le.fit_transform(sample[col].values)

In [175]:
sample[catg_feats] = sample[catg_feats].fillna('None')
sample[catg_feats].isnull().sum()

方角           0
建物構造         0
adr0         0
adr1         0
line0        0
station0     0
line1        0
station1     0
line2        0
station2     0
floorplan    0
dtype: int64

In [176]:
count_list = ['adr1','line0','station0','line1','station1','line2','station2','floorplan']

for col in count_list:
  count_mean = sample.groupby(col)['賃料'].count()
  sample[col + '_countall'] = sample[col].map(count_mean)

In [177]:
target_list = ['line0','station0','line1','station1','line2','station2','floorplan','建物構造','方角']

for col in target_list:
  target_mean = sample.groupby(col)['賃料'].mean()
  sample[col + '_groupby_targetmean'] = sample[col].map(target_mean)

In [178]:
station0_list = ['area','age','distance0','thisfloor','maxfloor']
for col in station0_list:
  station0_to_col = sample.groupby('station0')[col].mean()
  sample[col + '_to_station0'] = sample['station0'].map(station0_to_col)
  sample[col + '_compe_station0mean'] = sample[col] / sample[col + '_to_station0']

In [179]:
station1_list = ['area','age','distance1','thisfloor','maxfloor']
for col in station1_list:
  station1_to_col = sample.groupby('station1')[col].mean()
  sample[col + '_to_station1'] = sample['station1'].map(station1_to_col)
  sample[col + '_compe_station1mean'] = sample[col] / sample[col + '_to_station1']

In [180]:
station2_list = ['area','age','distance2','thisfloor','maxfloor']
for col in station2_list:
  station2_to_col = sample.groupby('station2')[col].mean()
  sample[col + '_to_station2'] = sample['station2'].map(station2_to_col)
  sample[col + '_compe_station2mean'] = sample[col] / sample[col + '_to_station2']

In [181]:
catg_x_num = ['area','age','thisfloor','maxfloor']
for col in catg_x_num:
  catg_x_num_mean = sample.groupby('adr0')[col].mean()
  sample[col + '_to_adr0'] = sample['adr0'].map(catg_x_num_mean)
  sample[col + '_compe_adr0mean'] = sample[col] / sample[col + '_to_adr0']

In [182]:
for col in catg_x_num:
  catg_x_num_mean = sample.groupby('adr1')[col].mean()
  sample[col + '_to_adr1'] = sample['adr1'].map(catg_x_num_mean)
  sample[col + '_compe_adr1mean'] = sample[col] / sample[col + '_to_adr1']

In [183]:
for col in catg_x_num:
  catg_x_num_mean = sample.groupby('line0')[col].mean()
  sample[col + '_to_line0'] = sample['line0'].map(catg_x_num_mean)
  sample[col + '_compe_line0mean'] = sample[col] / sample[col + '_to_line0']

In [184]:
for col in catg_x_num:
  catg_x_num_mean = sample.groupby('line1')[col].mean()
  sample[col + '_to_line1'] = sample['line1'].map(catg_x_num_mean)
  sample[col + '_compe_line1mean'] = sample[col] / sample[col + '_to_line1']

In [185]:
for col in catg_x_num:
  catg_x_num_mean = sample.groupby('line2')[col].mean()
  sample[col + '_to_line2'] = sample['line2'].map(catg_x_num_mean)
  sample[col + '_compe_line2mean'] = sample[col] / sample[col + '_to_line2']

In [186]:
for col in catg_x_num:
  catg_x_num_mean = sample.groupby('方角')[col].mean()
  sample[col + '_to_方角'] = sample['方角'].map(catg_x_num_mean)
  sample[col + '_compe_方角mean'] = sample[col] / sample[col + '_to_方角']

In [187]:
for col in catg_x_num:
  catg_x_num_mean = sample.groupby('建物構造')[col].mean()
  sample[col + '_to_建物構造'] = sample['建物構造'].map(catg_x_num_mean)
  sample[col + '_compe_建物構造mean'] = sample[col] / sample[col + '_to_建物構造']

In [188]:
for col in catg_x_num:
  catg_x_num_mean = sample.groupby('floorplan')[col].mean()
  sample[col + '_to_floorplan'] = sample['floorplan'].map(catg_x_num_mean)
  sample[col + '_compe_floorplanmean'] = sample[col] / sample[col + '_to_floorplan']

In [189]:
sample['Floor_area'] = sample['area'] / sample['Floor_total']
sample['high_this_area'] = sample['area'] * sample['thisfloor']
sample['high_max_area'] = sample['area'] * sample['maxfloor']

In [190]:
sample.drop(catg_feats,axis=1,inplace=True)

In [191]:
sample.drop(['id','Room'],axis=1,inplace=True)

In [192]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62732 entries, 0 to 62731
Columns: 290 entries, 賃料 to high_max_area
dtypes: float64(136), int64(154)
memory usage: 138.8 MB


In [193]:
sample.shape

(62732, 290)

In [194]:
sample.isnull().sum()

賃料                              31262
adr2                             2084
adr3                            37503
adr4                            40322
distance0                           0
                                ...  
maxfloor_to_floorplan               0
maxfloor_compe_floorplanmean       34
Floor_area                          0
high_this_area                      1
high_max_area                      34
Length: 290, dtype: int64

In [195]:
train_len = len(train)

In [196]:
long_train = sample.loc[:train_len-1]
long_test = sample.loc[train_len:]

In [197]:
print(sample.shape,long_train.shape,long_test.shape)

(62732, 290) (31470, 290) (31262, 290)


In [198]:
long_train.shape[0] + long_test.shape[0]

62732

In [199]:
long_train.to_csv(os.path.join(INPUT_DIR,'train_enginiring.csv'),index=False)
long_test.to_csv(os.path.join(INPUT_DIR,'test_enginiring.csv'),index=False)